# Analysing disability course in MS

This tutorial illustrates how to use the `pymsprog` package to study the evolution of disability in multiple sclerosis (MS) based on repeated assessments through time of an outcome measure (EDSS, NHPT, T25FW, SDMT; or any custom outcome measure). We'll start by illustrating the type of [input data](#input-data) needed, and by giving a [minimal working example](#minimal-example) to introduce the main function and facilitate the reading of the document. We'll then move on to a more detailed description of the different parameter configurations.

In [1]:
import pymsprog
print(pymsprog.__version__)

1.0.2


(input-data)=
## Input data

The data must be organised in a `pandas` `DataFrame` containing (at least) the following columns:

* Subject IDs;
* Visit dates;
* Outcome values.

The visits should be listed in chronological order (if they are not, `MSprog()` will sort them automatically before analysing them).

For relapsing-remitting MS patients, an additional `DataFrame` with the dates of relapses is needed to correctly assess progression and characterise progression events as relapse-associated or relapse-independent. The dataset should contain (at least) the following columns:

* Subject IDs;
* Visit dates.

In this tutorial, we will use toy data with artificially generated EDSS and SDMT assessments and relapse dates for a few hypothetical patients:

In [2]:
from pymsprog import load_toy_data
toydata_visits, toydata_relapses = load_toy_data()

print('\nVisits:')
print(toydata_visits.head())
print('\nRelapses:')
print(toydata_relapses.head())


Visits:
   id        date  EDSS  SDMT
0   1  2021-09-23   4.5    54
1   1  2021-11-03   4.5    54
2   1  2022-01-19   4.5    57
3   1  2022-04-27   4.5    55
4   1  2022-07-12   5.5    57

Relapses:
   id        date
0   2  2021-06-12
1   2  2022-10-25
2   3  2022-12-01
3   6  2022-12-18


(minimal-example)=
## Minimal example

The core block of `pymsprog` is the `MSprog()` function. Given data on visits and relapses in the form specified above, `MSprog()` analyses the disability evolution for each subject for the outcome of interest. We can test it on the EDSS toy data...

In [3]:
from pymsprog import MSprog
summary_edss, results_edss = MSprog(toydata_visits,  # data on visits
                         subj_col='id', value_col='EDSS', date_col='date',  # specify column names
                         outcome='edss',  # specify outcome type <--- EDSS
                         relapse=toydata_relapses) # data on relapses


---
Outcome: EDSS
Confirmation over: [84] days (-7 days, +730.5 days)
Baseline: fixed
Relapse influence (baseline): [30, 0] days
Relapse influence (event): [0, 0] days
Relapse influence (confirmation): [30, 0] days
Events detected: firstCDW
        
---
Total subjects: 6
---
Subjects with disability worsening: 3 (PIRA: 2; RAW: 1)


...or on the SDMT toy data:

In [4]:
summary_sdmt, results_sdmt = MSprog(toydata_visits,  # data on visits
                         subj_col='id', value_col='EDSS', date_col='date',  # specify column names
                         outcome='sdmt',  # specify outcome type <--- SDMT
                         relapse=toydata_relapses) # data on relapses


---
Outcome: SDMT
Confirmation over: [84] days (-7 days, +730.5 days)
Baseline: fixed
Relapse influence (baseline): [30, 0] days
Relapse influence (event): [0, 0] days
Relapse influence (confirmation): [30, 0] days
Events detected: firstCDW
        
---
Total subjects: 6
---
Subjects with disability worsening: 2 (PIRA: 2; RAW: 0)


The function prints concise info (the `verbose` argument can be used to control the amount of info printed out -- see [Printing progress info](#printing-progress-info) section below), and generates the following two `DataFrame`s:

<br />

1. event counts and event sequence (where relevant) for each subject. Here's what it looks like for the EDSS-based computation above:

In [5]:
print(summary_edss)

  event_sequence  CDW  RAW  PIRA  undef_CDW
1           PIRA    1    0     1          0
2            RAW    1    1     0          0
3                   0    0     0          0
4                   0    0     0          0
5           PIRA    1    0     1          0
6                   0    0     0          0


where: `event_sequence` where: `event_sequence`(\*) specifies the order of the events; the other columns count the events of each kind (improvement; relapse-associated worsening, [RAW](#raw); progression independent of relapse activity, [PIRA](#pira); and worsening that could not be classified as either RAW of PIRA with the available information). See {cite:t}`lublin2014,kappos2018`.

(\*) In this example, `event_sequence` can contain a maximum of one event, as we are not specifying the argument `event` - which defaults to `firstCDW` (only detect the first CDW event). For the same reason, `nevent` in `output$results` can only be 0 or 1 in this example. In a multiple-event setting (`event=multiple`), more than one event per subject can be detected. See [the relevant section](#multiple-events).

<br />

2. Extended info on each event for all subjects:

In [6]:
print(results_edss)

   id  nevent event_type  total_fu  time2event  bl2event  conf84  PIRA_conf84  \
0   1       1       PIRA       534       292.0     292.0       1            1   
1   2       1        RAW       730       198.0     198.0       1            0   
2   3       0                  491       491.0       NaN       0            0   
3   4       0                  586       586.0       NaN       0            0   
4   5       1       PIRA       637       140.0     140.0       1            1   
5   6       0                  491       491.0       NaN       0            0   

   sust_days  sust_last  
0        242          1  
1         84          0  
2          0          0  
3          0          0  
4        497          1  
5          0          0  


where: `nevent` is the cumulative event count for each subject; `event_type` characterises the event; `total_fu` is the number of days from start to end of follow-up; `time2event` is the number of days from start of follow-up to event(*); `bl2event` is the number of days from current baseline to event; `conf84` reports whether the event was confirmed over 84 days (12 weeks); `sust_days` is the number of days for which the event was sustained; `sust_last` reports whether the event was sustained until the last visit.

(*): For subjects with no confirmed events, `time2event` is the total follow-up length. To omit these subjects, set `include_stable=False` in `MSprog()`.

Several qualitative and quantitative options for analysing disability evolution are given as optional arguments of `MSprog()` that can be set by the user. In order to ensure reproducibility, the results should always be complemented by the set of criteria used to obtain them. In the following sections we will go into more detail about usage and best practices for the different options. Please refer to the documentation for a complete illustration of each of the function arguments and their default values.

(outcome)=
## Outcome

The `MSprog()` function detects the events sequentially by scanning the outcome values in chronological order, and each value is tested for its difference from the current reference value. Such difference is typically required to be larger than a threshold $\delta$, depending on outcome type and on the reference value $x$. If the `outcome` argument in `MSprog()` is set to `'edss'`, `'nhpt'`, `'t25fw'`, or `'sdmt'`, the following default settings are implemented {cite:p}`lorscheider2016,bosma2010,kalinowski2022,strober2019`:

* `outcome='edss'`: Expanded Disability Status Scale (EDSS). $\delta(x)=\begin{cases} 1.5 \quad \text{ if } x=0\\1 \quad\;\;\; \text{ if } 0 < x \leq 5\\0.5 \quad \text{ if } 5<x\leq 10\end{cases}$;

* `outcome='nhpt'`: Nine-Hole Peg Test (NHPT), for either the dominant or the non-dominant hand. $\delta=$ 20% of reference score;

* `outcome='t25fw'`: Timed 25-Foot Walk (T25FW). $\delta=$ 20% of reference score;

* `outcome='sdmt'`: Symbol Digit Modalities Test (SDMT). $\delta=$ either 3 points or 10% of reference score.

These default options are internally implemented by the `compute_delta()` function. For example, if the baseline EDSS score is 4, a valid worsening will correspond to an increase by (at least):

In [7]:
from pymsprog import compute_delta
print(compute_delta(4)) # default outcome measure is 'edss'

1.0


If the baseline T25FW score is 10, the minimum shift that is considered as a valid change will be:

In [8]:
print(compute_delta(10, outcome='t25fw'))

2.0


### Custom threshold values

The user can provide their own function to customise the computation of the threshold values, using the `delta_fun` argument of `MSprog()`. The provided function must take the baseline value as argument, and return the corresponding threshold. This applies to two scenarios.

1. **The outcome of interest is among the ones listed above, but we want to define the thresholds differently.** For example, we want to change the minimum $\delta$ for SDMT to, say, "either 4 points or 20% of the reference value". In this case, we would define:

In [9]:
def my_sdmt_delta(x):
    return min(4, x/5)
print('CUSTOM minimum valid change from baseline SDMT=50: ', my_sdmt_delta(50)) # my delta
print('DEFAULT minimum valid change from baseline SDMT=50: ', compute_delta(50, outcome='sdmt')) # default delta

CUSTOM minimum valid change from baseline SDMT=50:  4
DEFAULT minimum valid change from baseline SDMT=50:  3


To use our custom function, we can then set `delta_fun=my_sdmt_delta` in `MSprog()` when computing the disability events.

2. **The outcome of interest is not among the ones listed above.** In this case, we set `outcome=None`, and `delta_fun` as our desired custom-defined function. By default, higher values of a custom outcome are interpreted as a worsening. If the opposite direction of worsening is desired, set the `worsening` argument to `'decrease'`. Note that the `worsening` argument is only used by `MSprog()` when `outcome` is set to `None`. Otherwise, `worsening` is automatically set to `'increase'` if `outcome` is set to `'edss'`, `'nhpt'`, `'t25fw'`, and to `'decrease'` if `outcome` is set to `'sdmt'`.


(baseline-scheme)=
## Baseline scheme

The assessment of the disability evolution strongly depends on the choices made in defining the starting point, i.e., the baseline. Different behaviours are appropriate in different contexts. This aspect is controlled by the `baseline` argument in `MSprog()`. The following three alternative baseline schemes can be adopted.

* **Fixed baseline** (`baseline='fixed'`, default): the baseline visit is set to be the first available assessment.

* **Improvement-based roving baseline** (`baseline='roving_impr'`): the baseline visit is initially set as the first available assessment, then updated after each confirmed improvement. **This scheme is suitable for a first-CDW setting to discard fluctuations around the baseline {cite:p}`muller2023`, but not recommended for clinical trial data as it may break the randomisation**. The re-baseline procedure can be made finer through the `sub_threshold_rebl` argument in `MSprog()`: setting `sub_threshold_rebl='improvement'`, for instance, moves the reference value after *any* confirmed improvement, even if the difference from the current reference is smaller than the minimum $\delta$. In this case, the sub-threshold improvement events are used to move the baseline, but not listed in the event sequence.

* **Worsening-based roving baseline** (`baseline='roving_wors'`): the baseline visit is initially set as the first available assessment, then updated after each confirmed worsening. **This scheme is suitable when the endpoint of interest is a specific type of CDW**: for instance, when searching specifically for PIRA events, the reference should be moved after any previous RAW event; when searching specifically for RAW events, the reference should be moved after any previous PIRA event. The re-baseline procedure can be made finer through the `sub_threshold_rebl` argument in `MSprog()`: setting `sub_threshold_rebl='worsening'`, for instance, moves the reference value after *any* confirmed worsening, even if the difference from the current reference is smaller than the minimum $\delta$. In this case, the sub-threshold worsening events are used to move the baseline, but not listed in the event sequence.

* **Roving baseline** (`baseline='roving'`): the baseline visit is initially set as the first available assessment, then updated after each event. **This scheme is recommended in a "multiple events" setting** {cite:p}`kappos2018` (see example below), **but not recommended for clinical trial data as it may break the randomisation**. The re-baseline procedure can be made finer through the `sub_threshold_rebl` argument in `MSprog()`: setting `sub_threshold_rebl='event'`, for instance,  moves the reference value after *any* confirmed change, even if the difference from the current reference is smaller than the minimum $\delta$. In this case, the sub-threshold events are used to move the baseline, but not listed in the event sequence.

**NOTE.** If the baseline data is stored in a separate file, the user should merge it with the main `DataFrame` containing longitudinal visit data. This can be done by inserting the baseline date and outcome value for each subject at the beginning of the `DataFrame` (not necessarily next to the other visits from the same subject -- they will be grouped by subject ID). <span style="color: red;">The possibility to directly input a separate file for baseline data may be added in the future.</span>

<br />

When a roving baseline is used, the reference is moved after a previously detected event. The new reference may be set at the date of the previous event (`proceed_from='event'`), or at the first confirmation visit of such event (`proceed_from='firstconf'`, default).

If the baseline visit occurs in the vicinity of a clinical relapse, the reference value may be overestimated. The `relapse_to_bl` argument allows to specify the minimum accepted distance (in days) of the baseline visit from the onset of a previous relapse. For instance, if `baseline='fixed'` and `relapse_to_bl=30`, the baseline visit is chosen as the first available visit satisfying the requirement of "no relapses within the preceding 30 days". If two values are provided, they are interpreted as intervals before *and after* the event -- e.g., `relapse_to_bl=[30, 2]` implements the constraint of "no relapses within the preceding 30 days or within the following 2 days". If relapse **end dates** are available (may be provided as an additional column in the relapse file whose name is specified by argument `renddate_col` in `MSprog()`), the first value of `relapse_to_bl` is overwritten by the relapse duration, unless it was set to 0 (in which case it stays 0 and the baseline is **not** moved based on the influence of previous relapses). If the `baseline` argument is set to a roving scheme, all the above applies to every baseline change as well. 

On top of the chosen baseline scheme, *post-relapse re-baseline* {cite:p}`kappos2020` can be applied by setting `relapse_rebl=True` in `MSprog()`. If this is enabled, the onset of each relapse prompts a re-baseline to the next available visit following the relapse and out of its influence (as per `relapse_to_bl`). 

Due to fluctuations in the data, the new reference value following a re-baseline may fall at a local minimum or maximum of the disability trajectory. These may be excluded for consideration as baseline visits using the `skip_local_extrema` argument in `MSprog()`.

Finally, as an additional constraint, an updated baseline can be forced to have a score greater than or equal to the previous baseline by setting `bl_geq=True` in `MSprog()`.

As already mentioned above, extra caution should be used when applying any re-baseline scheme to randomised data, as moving the reference value based on post-randomisation events can introduce bias (especially if the occurrence of these events is influenced by the treatment). **For clinical trial data, general recommendation is to use a fixed baseline (at the time of randomisation).**


(multiple-events)=
### Multiple events

The `event` argument allows to specify which events to detect. By default, it is set to `'firstCDW'` (only detect the first CDW event). It can be set to `'multiple'` to sequentially detect all disability worsening or improvement events.

For example, extracting multiple EDSS events for subject `5` from `toydata_visits` with a **fixed** baseline would result in the following.

In [10]:
print('\nData:')
print(toydata_visits.loc[toydata_visits['id']==5, ['date', 'EDSS']].reset_index(drop=True)) # EDSS visits

_, results = MSprog(toydata_visits, 'id', 'EDSS', 'date', outcome='edss',
                    relapse=toydata_relapses, 
                    subjects=[5],
                    conf_days=12 * 7, proceed_from='firstconf',
                    event='multiple', baseline='fixed', 
                    include_dates=True, verbose=0)
print('\n---Results with fixed baseline:---')
print(results.drop(columns='id').set_index('nevent', drop=True).T) # results


Data:
         date  EDSS
0  2021-06-14   4.0
1  2021-09-12   4.0
2  2021-11-01   5.0
3  2022-01-17   5.5
4  2022-04-30   5.0
5  2022-07-06   5.5
6  2022-11-09   5.5
7  2022-12-15   5.0
8  2023-03-13   5.5

---Results with fixed baseline:---
nevent                         1                    2                    3
event_type                  PIRA                 PIRA                 PIRA
bldate       2021-06-14 00:00:00  2021-06-14 00:00:00  2021-06-14 00:00:00
date         2021-11-01 00:00:00  2022-04-30 00:00:00  2022-12-15 00:00:00
total_fu                     637                  637                  637
time2event                 140.0                320.0                549.0
bl2event                   140.0                320.0                549.0
conf84                         1                    1                    1
PIRA_conf84                    1                    1                    1
sust_days                    497                  317                   88
sust_la

With these settings, the EDSS worsening at visit `2` (EDSS=5.0, confirmed at visit `3`) does not trigger a re-baseline. The algorithm keeps searching for events from after the confirmation visit (`proceed_from='firstconf'`), evaluating the changes **relative to the original baseline** (EDSS=4.0, visit `0`). This results in two more "worsening events" detected at visits `4` (EDSS=5.0) and visit `7` (EDSS=5.0), when in reality the EDSS score is stable. This mistake is avoided by adopting a **roving** baseline scheme:

In [11]:
_, results = MSprog(toydata_visits, 'id', 'EDSS', 'date', outcome='edss',
                    relapse=toydata_relapses, 
                    subjects=[5],
                    conf_days=12 * 7, proceed_from='firstconf',
                    event='multiple', baseline='roving',  # <-----
                    include_dates=True, verbose=0)
print('\n---Results with roving baseline:---')
print(results.drop(columns='id').set_index('nevent', drop=True).T) # results


---Results with roving baseline:---
nevent                         1
event_type                  PIRA
bldate       2021-06-14 00:00:00
date         2021-11-01 00:00:00
total_fu                     637
time2event                 140.0
bl2event                   140.0
conf84                         1
PIRA_conf84                    1
sust_days                    497
sust_last                      1


The baseline is now moved to the confirmation visit of the first worsening event (PIRA at visit `3`, EDSS=5.5). No further confirmed changes are detected with respect to this baseline.
<br />

Other valid values for the `event` argument are:

* `'first'`: only the very first event -- improvement or worsening;
* `'firsteach'`: first improvement *and* first worsening, in chronological order;
* `'firstCDWtype'`: first CDW of each kind -- PIRA, RAW, and undefined, in chronological order;
* `'firstPIRA'`: first PIRA event;
* `'firstRAW'`: first RAW event.

Note: if, for example, `'event=firstPIRA'`, all non-PIRA events *preceding* the first PIRA are actually detected -- although not reported. This allows to combine this scenario with a roving baseline scheme, where the baseline is moved after every confirmed non-PIRA event. For instance, if a RAW event occurs first, the subsequent worsening would be counted from the re-baselined outcome value after the RAW.



(event-confirmation)=
## Event confirmation

An event is only validated if the change in the outcome value from the current baseline is maintained **up to**(\*) a subsequent *confirmation visit* at a pre-specified distance from the event {cite:p}`ontaneda2017`. The event is confirmed if the difference in the outcome value from the baseline score remains above-[threshold](#outcome) at all assessments up to the confirmation visit. For example, with the default EDSS thresholds, an increase in EDSS from 4 points to 6 points is confirmed if EDSS=5 at all subsequent visits; it is not confirmed if EDSS=4.5 at all subsequent visits.

The chosen confirmation period depends on the type of study and on the frequency of visits, and can be set in `MSprog()` by using the `conf_days` argument. A tolerance interval around `conf_days` can be specified using the `conf_tol_days` argument, given as a sequence of two integers (left and right tolerance)(\**). Setting the right end of the interval to `Inf` allows event confirmation to occur at any visit *after* `conf_days`. Let's see two examples.

i. A common setting for **clinical trial data** would be: `conf_days=7*12`, `conf_tol_days=[0, np.inf]`, 
i.e., "confirmation over 12 or more weeks".

ii. A common setting for **observational data** would be: `conf_days=7*24`, `conf_tol_days=[0, 7*12]`, 
i.e., the confirmation visit must lie between 24 weeks after the event, and 36 weeks after the event.

### Additional options

* If `conf_days` is specified as a sequence of values (e.g., `conf_days=[7*12, 7*24]`), events are retained if confirmed by *at least one visit* falling within any of the specified periods (here, 12 **or** 24 weeks with their relative tolerance interval)(\***). The results table will report whether an event was confirmed in each of the specified periods, see example [below](#example).

* If the confirmation visit occurs in the vicinity of a clinical relapse, it is typically considered invalid. The `relapse_to_conf` argument allows to specify the minimum accepted distance (in days) of a confirmation visit from the onset of a previous relapse. For example, `relapse_to_conf=30` implements the constraint "no relapses within the preceding 30 days" for a visit to be used as confirmation. If two values are provided, they are interpreted as intervals before *and after* the event -- e.g., `relapse_to_conf=[30, 2]` implements the constraint "no relapses within the preceding 30 days or within the following 2 days". If relapse **end dates** are available (may be provided as an additional column in the relapse file whose name is specified by argument `renddate_col` in `MSprog()`), the first value of `relapse_to_conf` is overwritten by the relapse duration, unless it was set to 0 (in which case it stays 0 and confirmation visits are **not** discarded based on the influence of previous relapses).

* By default, a disability worsening occurring at the last available assessment is ignored by `MSprog()`. This behaviour can be changed using the `impute_last_visit` argument. If set to `TRUE`, any disability worsening occurring at the last visit is retained even though unconfirmed. If set to a numeric value N, unconfirmed worsening events are included only if occurring within N days of follow up (e.g., in case of early discontinuation).

* In scenarios (e.g., a clinical trial) where disability course is compared between two cohorts with different relapse rates, there may be an imbalance in visit frequency between the two groups, due to unscheduled visits after relapses. This implies a higher probability of detecting outcome changes in the group with more assessments. Such issue is sometimes addressed by only using scheduled visits as confirmation visits {cite:p}`hauser2017`. It is possible to implement this with `MSprog()` by including an additional column in the longitudinal data, specifying which visits can (`True`) or cannot (`False`) be used as confirmation visits. The name of such column must be specified using the `validconf_col` argument -- for example, if the database contains a column `'scheduled'` tracking scheduled visits, unscheduled visits can be excluded from consideration as confirmation visits by setting `validconf_col='scheduled'`in `MSprog()`.


(\*) The value change from baseline must also be maintained at all intermediate visits between the event and the confirmation visit. This behaviour may be changed by setting the `check_intermediate` argument to `False` (in this case, the change only needs to be confirmed **at** the designated confirmation visit). We do not recommend this, as it may lead to including random fluctuations as events. We included this option to provide the possibility of replicating the results from previous studies that used this rationale.

(\**) `conf_tol_days` can also be specified as a single integer if the same tolerance on left and right is desired.

(\***) An event is only confirmed if the value change from baseline is maintained **at all visits up to the confirmation visit**. So an event can only be "confirmed over 24 weeks" and *not* "confirmed over 12 weeks" if there are no valid confirmation visits falling within the 12-week window (unless `check_intermediate=FALSE`).

### Sustained worsening or improvement

In addition to event confirmation, some studies require events to be *sustained* for a specified period of time. The `require_sust_days` argument allows to specify, if desired, the length of such period. For example, if `require_sust_days=7*48`, an event is only retained if the change in the outcome measure from the current baseline is confirmed at *all* visits falling within the following 48 weeks. If the event is sustained for the entire follow-up, it is retained even if the follow-up period ends <48 weeks after the event. Setting `require_sust_days=Inf`, events are retained only when sustained for the entire follow-up duration.

The `require_sust_days` argument may be of use in the following scenarios.

* Suppose the maximum follow-up in the population is 96 weeks excluding the baseline visit. If one wants to detect CDWs sustained over the whole follow-up, coding it as "worsening confirmed over 96 weeks or more" (`conf_days=7*96`, `conf_tol_days=c(0, Inf)`) would discard CDWs of patients whose follow-up is shorter than 96 weeks. In a scenario where patients have different follow-up lengths, the `require_sust_days` argument offers a more reliable way of implementing this, e.g., as "worsening confirmed over 12 weeks or more and sustained for the whole follow-up period" (`conf_days=7*12`, `conf_tol_days=c(0, Inf)`, `require_sust_days=Inf`). 

* In observational data with coarsely- and unevenly-spaced visits, the `require_sust_days` argument may be used to detect sustained CDW (e.g., `require_sust_days=7*96`) while still requiring the presence of a confirmation visit within a specified interval from the event (e.g., `conf_days=7*12`, `conf_tol_days=c(0,7*12)`). 

* Sustained disability worsening can be required to exclude transient RAW events (see example [below](#example)).


(example)=
### Example

Let's look at subject `2` from `toydata_visits`:

In [12]:
print('\nVisits:')
print(toydata_visits.loc[toydata_visits['id']==2, ['date', 'EDSS']].reset_index(drop=True)) # EDSS visits

print('\nRelapses:')
print(toydata_relapses[toydata_relapses['id']==2].reset_index(drop=True)) # relapses


Visits:
         date  EDSS
0  2020-11-26   4.0
1  2020-12-30   4.0
2  2021-03-24   4.5
3  2021-06-12   5.5
4  2021-09-04   5.0
5  2021-12-02   4.5
6  2022-02-23   4.5
7  2022-05-19   6.0
8  2022-08-28   6.0
9  2022-11-26   6.0

Relapses:
   id        date
0   2  2021-06-12
1   2  2022-10-25


The following code detects 12- or 24-week (84- or 168-day) confirmed events for subject `2`.

In [13]:
_, results = MSprog(toydata_visits, subj_col='id', value_col='EDSS', date_col='date', outcome='edss',
                    relapse=toydata_relapses, subjects=[2],
                    conf_days=[12*7, 24*7], event='multiple', baseline='roving', 
                    relapse_indep={'prec': (0, None), 'event': (None, None), 'conf': (None, 0)},
                    verbose=0)
print('\n---Results:---')
print(results.drop(columns='id').set_index('nevent', drop=True).T) # results


---Results:---
nevent            1      2
event_type      RAW   PIRA
total_fu        730    730
time2event    198.0  539.0
bl2event      198.0  257.0
conf84            1      1
conf168           0      1
PIRA_conf84       0      1
PIRA_conf168      0      0
sust_days        84    191
sust_last         0      1


The `relapse_indep` argument specifies the relapse-free intervals for a CDW event to be classified as PIRA, see [the relevant section](#pira). In this case, a high-specificity definition of PIRA is used {cite:p}`muller2023` that requires an absence of relapses between baseline and confirmation. The resulting events are

1. a 12-week-confirmed RAW;
2. a 12-week-confirmed PIRA.

Event #1 was confirmed over 12 weeks but not over 24 weeks, i.e., 168 days (`conf168` is `0`). Event #2 was also confirmed over 24 weeks (`conf168` is `1`) as a CDW. However, since a relapse occurred before the 24-week confirmation, it is not confirmed as PIRA over 24 weeks (`PIRA_conf168` is `0`).

The RAW event found for subject `2` constitutes a *transient* accumulation of disability. Such events can be excluded from the `MSprog()` output by requiring that each event be sustained for at least a certain amount of time (specified by argument `require_sust_days`). For instance:

In [14]:
_, results = MSprog(toydata_visits, subj_col='id', value_col='EDSS', date_col='date', outcome='edss',
                    relapse=toydata_relapses, subjects=[2],
                    conf_days=[12*7, 24*7], event='multiple', baseline='roving', 
                    require_sust_days=48*7,  # <---
                    relapse_indep={'prec': (0, None), 'event': (None, None), 'conf': (None, 0)},
                    verbose=0)
print('\n---Results with require_sust_days=48*7:---')
print(results.drop(columns='id').set_index('nevent', drop=True).T) # results


---Results with require_sust_days=48*7:---
nevent            1
event_type     PIRA
total_fu        730
time2event    539.0
bl2event      257.0
conf84            1
conf168           1
PIRA_conf84       1
PIRA_conf168      0
sust_days       191
sust_last         1


A more detailed report of the event detection process in each of the cases examined above may be visualized by re-running the above code snippets with `verbose=2`, see [Printing progress info](#printing-progress-info) section below.




## Classification of CDW events

When relevant, relapse data is to be provided using the optional `relapse` argument in `MSprog()`. It should be given as a `DataFrame` containing subject IDs and relapse onset dates (see section [Input data](#input-data))(\*).

(\*) If the names of "subject ID" and "date" columns in the relapse database are different from the main database, they must be specified using arguments `rsubj_col` and `rdate_col`.

(raw)=
### RAW

RAW events are typically defined as CDW events occurring within a specified interval from the onset of a relapse. The length (in days) of such interval can be specified using the `relapse_assoc` argument in `MSprog()`. Common values are 30 or 90 days. Additionally, one may also provide a maximum distance from relapses whose onset is *after* the event. The logic is that the reported onset date of a relapse may be slightly delayed. The examples below illustrate the usage of the `relapse_assoc` argument:

* `relapse_assoc=90` (equivalent to `relapse_assoc=[90, 0]`): RAW = "a CDW event occurring within 90 days after the onset of a relapse";
* `relapse_assoc=[90, 7]`: RAW = "a CDW event occurring no more than 90 days after and no more than 7 days before the onset of a relapse".

(pira)=
### PIRA

In the literature, PIRA is typically defined by requiring an absence of relapses within appropriate intervals based on the dates of the baseline visit, the worsening event, and the confirmation visit {cite:p}`kappos2020,cagol2022,muller2023`. The `relapse_indep` argument in `MSprog()` allows to specify custom relapse-free intervals. The argument must be provided in the form of a dictionary as follows:

`{'prec': (p0, p1), 'event': (e0, e1), 'conf': (c0, c1)}`,
    
The dictionary specifies the intervals \[`p0`, `p1`\], \[`e0`, `e1`\], and \[`c0`, `c1`\] around (any subset of) the three checkpoints:

- `'prec'`: a visit preceding the event -- see below;
- `'event'`: the disability worsening event;
- `'conf'`: the first available confirmation visit.

The dictionary can also optionally contain a key-value pair specifying how to choose `'prec'`:

- `'prec_type': 'baseline'` → 'prec' is the current baseline;
- `'prec_type': 'last'` → 'prec' is the last visit before the event;
- `'prec_type': 'last_lower'` → 'prec' is the last pre-worsening visit:
  `i` such that `outcome[event] - outcome[i] >= delta_fun(outcome[i])` (if `worsening='increase'`, the opposite otherwise)
  and same for the confirmation visit.

If `'prec_type'` is not in the dictionary keys, `'prec'` is assumed to be the current baseline.

If both ends of an interval are 0 (e.g., if both `p0=0` and `p1=0`), the checkpoint is ignored.
If the right end is `None`, the interval is assumed to extend up to the left end of the next interval.
If the left end is `None`, the interval is assumed to extend up to the right end of the previous interval.

For example, in {cite:t}`muller2023`, the authors recommend an absence of relapses during the 90 days before and 30 days after the event, and during the 90 days before and 30 days after the confirmation visit for a CDW event to be considered as PIRA. This translates into: `p0 = 0`, `p1 = 0`, `e0 = 90`, `e1 = 30`, `c0 = 90`, `c1 = 30`. When a high specificity is desired, they recommend an absence of relapses in the whole period between the reference visit and the confirmation visit, which is implemented by setting: `p0 = 0`, `p1 = None`, `e0 = None`, `e1 = None`, `c0 = None`, `c1 = 0`. 

<br />



## `MSprog()` outputs

### What to include in results

The results `DataFrame` provides extended info on each event for all subjects. The following arguments can be used to control the information included.

* `include_dates`: if `True`, the results will include the dates of baseline and event;
* `include_value`: if `True`, the results will include the outcome value at baseline and event;
* `include_stable`: if `True`, subjects with no confirmed events are included in the results, with `time2event` = total follow up.


For example:

In [15]:
summary, results = MSprog(toydata_visits,
                         subj_col='id', value_col='EDSS', date_col='date',
                         outcome='edss',
                         relapse=toydata_relapses,
                         include_dates=True, include_value=True, include_stable=False,
                         verbose=0)
print(results.T)

                               0                    1                    2
id                             1                    2                    5
nevent                         1                    1                    1
event_type                  PIRA                  RAW                 PIRA
bldate       2021-09-23 00:00:00  2020-11-26 00:00:00  2021-06-14 00:00:00
blvalue                      4.5                  4.0                  4.0
date         2022-07-12 00:00:00  2021-06-12 00:00:00  2021-11-01 00:00:00
value                        5.5                  5.5                  5.0
total_fu                     534                  730                  637
time2event                 292.0                198.0                140.0
bl2event                   292.0                198.0                140.0
conf84                         1                    1                    1
PIRA_conf84                    1                    0                    1
sust_days                

(printing-progress-info)=
### Printing progress info

The `verbose` argument controls the amount of info printed out by the `MSprog()` function. When setting `verbose=0`, the function prints no info. When setting `verbose=1` (default), the function only prints out concise info. When setting `verbose=2`, the function prints out an extended log of the ongoing computations. 

For further insight into the event detection process, the `return_unconfirmed` argument allows to also return all score changes from baseline that were not identified as valid events (e.g., not confirmed).

See the example below.

In [16]:
summary, results, unconfirmed = MSprog(toydata_visits,
                          subj_col='id', value_col='EDSS', date_col='date',
                          outcome='edss',
                          event='multiple', baseline='roving',
                          relapse=toydata_relapses,
                          return_unconfirmed=True,  # <---
                          verbose=2)  # <---


Subject #1: 8 visits, 0 relapses
Baseline at visit no.1
Searching for events from visit no.2 on
EDSS change at visit no.5 (2022-07-12); potential confirmation visits available: no.6, 7, 8
EDSS PIRA (visit no.5, 2022-07-12) confirmed at 84 weeks, sustained up to visit no.8 (2023-03-11)
Baseline at visit no.6
Searching for events from visit no.7 on
No EDSS change in any subsequent visit: end process
Event sequence: PIRA

Subject #2: 10 visits, 2 relapses
Baseline at visit no.1
Searching for events from visit no.2 on
EDSS change at visit no.4 (2021-06-12); potential confirmation visits available: no.5, 6, 7, 8, 9, 10
EDSS RAW (visit no.4, 2021-06-12) confirmed at 84 weeks, sustained up to visit no.5 (2021-09-04)
Baseline at visit no.5
Searching for events from visit no.6 on
EDSS change at visit no.8 (2022-05-19); potential confirmation visits available: no.9, 10
EDSS PIRA (visit no.8, 2022-05-19) confirmed at 84 weeks, sustained up to visit no.10 (2022-11-26)
Baseline at visit no.9
Searc

<br/>

Let's visualise the unconfirmed events:

In [17]:
print('---Unconfirmed events:---')
print(unconfirmed)

---Unconfirmed events:---
   id       date  value     bldate  blvalue  closest_rel-  closest_rel+
0   3 2021-12-01    4.5 2021-10-18      3.5           inf         365.0
1   5 2022-04-30    5.0 2022-01-17      5.5           inf           inf
2   5 2022-12-15    5.0 2022-01-17      5.5           inf           inf
3   6 2022-12-22    6.0 2022-09-03      4.5           4.0           inf
4   6 2023-02-21    6.0 2022-09-03      4.5          65.0           inf


## Time to disability milestone

Instead of studying disability course with respect to a baseline value, one can focus on the time taken to reach a specific disability milestone (e.g., EDSS $\geq$ 6.0). This can be computed using another function from the `msprog` package, `value_milestone()`.

The following code detects the time to EDSS $\geq$ 6.0 for each subject in the toy data. 

In [18]:
from pymsprog import value_milestone

vm = value_milestone(toydata_visits, milestone=6,
                 subj_col='id', value_col='EDSS', date_col='date', outcome='edss',
                 relapse=toydata_relapses,
                 verbose=0)

The function returns a `DataFrame` with one row per subject:

In [19]:
print(vm)

        date  EDSS  time2event  observed
1 2023-03-11   NaN       534.0         0
2 2022-05-19   6.0       539.0         1
3 2023-02-21   NaN       491.0         0
4 2023-04-27   NaN       586.0         0
5 2023-03-13   NaN       637.0         0
6 2023-02-21   NaN       491.0         0


where: `'date'` contains the date of the first confirmed EDSS $\geq$ 6.0 (or last date of follow-up if milestone is not reached or not confirmed); `'EDSS'` contains the first EDSS value $\geq$ 6.0, if present, otherwise no value; `'time2event'` contains the time to reach EDSS $\geq$ 6.0 (or total follow-up length if not reached or not confirmed); `'observed'` indicates whether EDSS $\geq$ 6.0 was reached (1) or not (0).

Several arguments controlling the behaviour of the `MSprog()` function are also available for the `value_milestone()` function (e.g., `require_sust_days`, `impute_last_visit`...). Please refer to the documentation for a complete illustration of each of the function arguments and their default values.

## References
```{bibliography}
:style: plain
:filter: cited
```